# MIP

In [10]:
from pulp import *

# Read the file and initialize the variables
with open("../data/inst02.dat", "r") as file:
    lines = file.readlines()

# Create a new model
model = LpProblem("CourierRoutingProblem", LpMinimize)

In [11]:
## Parse the values from the file
m = int(lines[0].strip())  # Number of couriers
n = int(lines[1].strip())  # Number of items
l_values = list(map(int, lines[2].split()))  # Maximum load for each courier
s_values = list(map(int, lines[3].split()))  # Size of each item
D_values = [list(map(int, line.split())) for line in lines[4:]]  # Distance matrix
#l = LpVariable.dicts("l", range(m), lowBound=0, cat="Integer")  # Maximum load decision variables
#for i in range(m):
#    l[i].bounds = (l_values[i], l_values[i])
#    model.addConstraint(l[i] == l_values[i])
#
#
#s = LpVariable.dicts("s", range(n), lowBound=0, cat="Integer")  # Size of each item decision variables
#for i in range(n):
#    s[i].bounds = (s_values[i], s_values[i])
#    model.addConstraint(s[i] == s_values[i])
#
#
#D = LpVariable.dicts("D", (range(n + 1), range(n + 1)), lowBound=0, cat="Integer")  # Distance matrix decision variables
#
#for i in range(n + 1):
#    for j in range(n + 1):
#        D[i][j].bounds = (D_values[i][j], D_values[i][j])
#        #D[i][j].setInitialValue(D_values[i][j])
#        model.addConstraint(D[i][j] == D_values[i][j])
#
#routes = LpVariable.dicts("routes", (range(n+1), range(n + 1)), lowBound=0, upBound=m, cat="Integer")  # Routes for each courier OLD MODEL
routes = [LpVariable.dicts(f"routes_{i+1}", (range(n+1), range(n + 1)), cat = "Binary") for i in range(m)]
y = [ LpVariable(f"y_{d}", cat="Binary") for d in range(m) ]




In [12]:
# objective function 
objective = [lpSum([D_values[t][j] * routes[i][t][j] for t in range(n+1) for j in range(n+1)]) for i in range(m)]


### CONSTRAINTS

In [13]:

model += lpSum(routes) == (n+m)

#No 1 values on diagonals
model += lpSum([routes[d][i][i] for d in range(m) for i in range(n+1)]) == 0

#Exactly 1 value in the same columns for each courier except last column
for j in range(n):
    model += lpSum([routes[d][i][j] for d in range(m) for i in range(n+1)]) == 1

#Exactly 1 value in the same row for each courier except last row
for i in range(n):
    model += lpSum([routes[d][i][j] for d in range(m) for j in range(n+1)]) == 1

#Exactly 1 value in the last row for each courier   
#model += lpSum([routes[d][n][j]  for d in range(m) for j in range(n+1)]) == m

#Exactly 1 value in the last column for each courier
#model += lpSum([routes[d][i][n]   for d in range(m) for i in range(n+1)]) == m

for d in range(m):

    #At least 1 value for each courier
    model += lpSum(routes[d]) >= 2

    #Exactly 1 value in the last row for each courier   
    model += lpSum([routes[d][n][j]  for j in range(n+1)]) == 1

    #Exactly 1 value in the last column for each courier
    model += lpSum([routes[d][i][n]  for i in range(n+1)]) == 1

    #Loads constraint
    model += lpSum([lpSum(routes[d][i]) * s_values[i] for i in range(n)]) <= l_values[d]

    s = lpSum([lpSum([ routes[d][i][j]*(i+1) for j in range(n)]) for i in range(n)]) - lpSum([lpSum([ routes[d][j][i]* (i+1) for j in range(n)])  for i in range(n)])
    sl = lpSum([lpSum([ routes[d][i][j]*(i+1) for j in range(n)]) for i in range(n)])
    sr = lpSum([lpSum([ routes[d][j][i]* (i+1) for j in range(n)])  for i in range(n)])
    # Add additional constraints based on the if-then condition
    

    # Add constraints if the condition is met (y = 1)
    model+= y[d] >= sr/sum([i for i in range(n)]), f"random constraint_{d}"
    model += y[d] <= sr
    model += s>= (1 * y[d])
    #model += s>= (-1 * y[d])
    model += s
    model += y[d]



        
    #The path must be coherent => there should exist a coherent path from the first destination point to the origin
    #It's implemented by verifying that the sum of the values on the row 1 is equal to the sum of values in the comumn 1 => 
    #if there's a row in the ith row there should be a 1 also in the ith column and viceversa
    for i in range(n+1):
        model += ( lpSum([routes[d][i][a] for a in range(n+1) ]) == lpSum([routes[d][t][i] for t in range(n+1)]) )


    for i in range(n):
        for j in range(n):
            ##Couriers cannot go back to a destination point already visited, except if it is the origin point
            #Example   o -> 1   1 -> is allowed
            model += lpSum([routes[d][i][j],routes[d][j][i]]) <=1


#TODO Symmetry breaking contraints?




In [14]:
maximum = LpVariable("maximum", lowBound=0, cat="Integer")

for i in range(len(objective)):
    model.addConstraint(constraint = (maximum>=objective[i])) 

model.setObjective(maximum)

# Solve the model
model.solve()

# The status of the solution is printed to the screen
print("Status:", LpStatus[model.status])

if model.status == LpStatusOptimal:
    length = model.objective.value()
   
else:
    print("No solution found.")

Status: Optimal


In [15]:
#Print results
routes_values = []
for d in range(m):
    r = [n+1] * n
    pos = 0
    index = n
    while True: 
        index = [routes[d][index][j].varValue for j in range(n+1)].index(1.0)
        r[pos] = index+1
        pos+=1
        if(index == n):
            routes_values.append(r)
            break

print("---ROUTES---")
print(routes_values)

print("---DISTANCES---")
print([sum([D_values[t][j] * routes[i][t][j].varValue for t in range(n+1) for j in range(n+1)]) for i in range(m)])
print("---ROUTES MASK ---")
for d in range(m):
    print(f"---COURIER {d+1}---")
    for i in range(n+1):
        print([routes[d][i][j].varValue for j in range(n + 1)])
        
print("---MAXIMUM---")
print(length)

---ROUTES---
[[1, 10, 10, 10, 10, 10, 10, 10, 10], [5, 10, 10, 10, 10, 10, 10, 10, 10], [7, 2, 10, 10, 10, 10, 10, 10, 10], [4, 10, 10, 10, 10, 10, 10, 10, 10], [9, 3, 10, 10, 10, 10, 10, 10, 10], [8, 6, 10, 10, 10, 10, 10, 10, 10]]
---DISTANCES---
[174.0, 192.0, 226.0, 188.0, 189.0, 199.0]
---ROUTES MASK ---
---COURIER 1---
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
---COURIER 2---
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

In [16]:
for d in range(m):
    print(y[d].varValue)

0.0
0.0
1.0
0.0
1.0
1.0


In [17]:
import pulp

# Create a PULP model
model = pulp.LpProblem("If-Then Example", pulp.LpMinimize)

# Define variables
x = pulp.LpVariable("x", lowBound=0, upBound=10)  # Variable x
y = pulp.LpVariable("y", cat="Binary")  # Binary variable representing the condition

# Define binary variables for the outcomes of the if-then statement
outcome_1 = pulp.LpVariable("outcome_1", cat="Binary")
outcome_2 = pulp.LpVariable("outcome_2", cat="Binary")

# Add constraints to enforce the if-then logic
model.addConstraint(x <= 5 + (10 - 5) * y)  # If y is 1 (True), x <= 10, otherwise x <= 5
model.addConstraint(outcome_1 >= y)  # outcome_1 = 1 (True) if y is 1 (True), otherwise outcome_1 = 0
model.addConstraint(outcome_2 >= 1 - y)  # outcome_2 = 1 (True) if y is 0 (False), otherwise outcome_2 = 0

# Objective function
model.setObjective(outcome_1 + outcome_2)

# Solve the model
model.solve()

# Print the solution
print("x =", pulp.value(x))
print("y =", pulp.value(y))
print("outcome_1 =", pulp.value(outcome_1))
print("outcome_2 =", pulp.value(outcome_2))


x = 0.0
y = 0.0
outcome_1 = 0.0
outcome_2 = 1.0


In [18]:
from pulp import LpProblem, LpVariable, LpMinimize, LpConstraint, LpStatusOptimal

# Create a new LP problem
problem = LpProblem("conditional_constraint", LpMinimize)

# Create variables
a = LpVariable("a", lowBound=0)
x = LpVariable("x", lowBound=0)

# Define the constraint
condition = (a >= 3)
constraint = LpConstraint(e=condition, sense=1, rhs=0, name="conditional_constraint")
problem += constraint

# Set the objective function
problem += x

# Solve the problem
problem.solve()

# Check the status of the solution
if problem.status == LpStatusOptimal:
    # Print the optimal solution
    print("Optimal solution:")
    print("a =", a.value())
    print("x =", x.value())
else:
    print("Problem status:", problem.status)


Optimal solution:
a = 3.0
x = 0.0
